In [1]:
#dependences
import tweepy
import pandas as pd
import numpy as np
import json
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timezone

In [2]:
#import vader for sentiment analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

In [3]:
# Twitter API Keys
consumer_key = "r13GARnSlIYpn4rT8Cky7lggU"
consumer_secret = "jNzYyOlz2cKvN5hqTetJGpa9C2HxeTGrq6FLb6J1VlZRwkFMg8"
access_token = "365121943-MtrTOA6FQ7ZXgkDM70f4fbrvmjI5vtucwok4nd8U"
access_token_secret = "7LiSAbsWOyLjpagesrBduFvL82NcP4rrdodlMfzCM9Q0r"

In [4]:
#autheticate
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, parser=tweepy.parsers.JSONParser())

In [5]:
#make dictionary of twitter SNs, network names, and colors for graphs
news_labels = {'CNN': 
                   {'name': 'CNN', 
                    'color': 'red'}, 
               'BBCWorld': 
                   {'name': 'BBC World News', 
                    'color': 'blue'}, 
               'CBSNews': 
                   {'name': 'CBS', 
                    'color': 'lightblue'},
               'nytimes': 
                   {'name': 'New York Times', 
                    'color': 'green'}, 
               'FoxNews': 
                   {'name': 'Fox', 
                    'color': 'yellow'}}

In [6]:
news_dict = {
    "Screen Name": [],
    "Tweet": [],
    "Date": [],
    "Compound Score": [],
    "Positive Score": [],
    "Negative Score": [],
    "Neutral Score": [],
    "Tweets Ago": []
}

In [7]:
# loop through SNs in dictionary
for news_org in news_labels.keys():
    tweets_ago = 1 #counts tweets 1 - most current 100 - least current, resets for each news_org
    for x in range(5): # loops through 5 pages of 20 tweets
        if x == 0:
            response = api.user_timeline(news_org, count = 20) #most recent 20 tweets
        else:
            response = api.user_timeline(news_org, max_id = end_id-1 , count = 20) # next 20 most recent tweets each time
        for tweet in response: # disects each tweet and pulls necessary info
            text = tweet['text'] #tweet text
            date = tweet['created_at'] #time created
            convert_date = datetime.strptime(date, "%a %b %d %H:%M:%S %z %Y") #converts date to sortable time
            user = tweet['user']['screen_name'] #SN of user
            end_id = tweet['id'] # gets id of tweet to be used in loop for next 20 results
            scores = analyzer.polarity_scores(text) # scores the text in tweet using VADER
            news_dict['Screen Name'].append(user) # adds SN to dict
            news_dict['Tweet'].append(text) # adds tweet text to dict
            news_dict['Date'].append(convert_date) # adds converted date to dict
            news_dict['Compound Score'].append(scores['compound']) 
            news_dict['Positive Score'].append(scores['pos'])
            news_dict['Negative Score'].append(scores['neg'])
            news_dict['Neutral Score'].append(scores['neu'])
            news_dict['Tweets Ago'].append(tweets_ago)
            tweets_ago += 1 # sets tweets ago count for next record

In [9]:
#converts to dataframe
news_df = pd.DataFrame(news_dict)

news_df.head()

,Compound Score,Date,Negative Score,Neutral Score,Positive Score,Screen Name,Tweet,Tweets Ago
0,-0.2732,2018-03-06 23:49:07+00:00,0.130,0.870,0.000,CNN,Presidential seal on Trump golf tees? A new lo...,1
1,-0.5267,2018-03-06 23:41:06+00:00,0.185,0.815,0.000,CNN,A group of senators are requesting an investig...,2
2,-0.5267,2018-03-06 23:38:01+00:00,0.175,0.825,0.000,CNN,"Jeffrey Toobin: ""Gary Cohn made a big show of ...",3
3,-0.1531,2018-03-06 23:36:04+00:00,0.082,0.865,0.053,CNN,"""It is laughable to argue an officer should go...",4
4,0.0000,2018-03-06 23:31:07+00:00,0.000,1.000,0.000,CNN,White House chief of staff John Kelly listens ...,5


In [11]:
len(news_df)

500

In [16]:
news_df.to_csv('Distinguishing-News-Sentiments/Output_CSVs/news_analysis.csv')


FileNotFoundError: [Errno 2] No such file or directory: 'Distinguishing-News-Sentiments/Output_CSVs/news_analysis.csv'